# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

[2025-05-08 00:36:39] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', chat_template=None, completion_template=None, is_embedding=False, revision=None, host='0.0.0.0', port=35581, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=281543259, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_

[2025-05-08 00:36:49] Attention backend not set. Use fa3 backend by default.
[2025-05-08 00:36:49] Init torch distributed begin.


[2025-05-08 00:36:49] Init torch distributed ends. mem usage=0.00 GB
[2025-05-08 00:36:49] Load weight begin. avail mem=53.74 GB


[2025-05-08 00:36:50] Ignore import error when loading sglang.srt.models.llama4. 


[2025-05-08 00:36:50] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


...

[2025-05-08 00:36:53] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-05-08 00:36:54] INFO:     Started server process [3577426]
[2025-05-08 00:36:54] INFO:     Waiting for application startup.
[2025-05-08 00:36:54] INFO:     Application startup complete.
[2025-05-08 00:36:54] INFO:     Uvicorn running on http://0.0.0.0:35581 (Press CTRL+C to quit)


[2025-05-08 00:36:55] INFO:     127.0.0.1:57240 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-08 00:36:55] INFO:     127.0.0.1:57246 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-08 00:36:55] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 00:36:57] INFO:     127.0.0.1:57256 - "POST /generate HTTP/1.1" 200 OK
[2025-05-08 00:36:57] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Please generate the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-08 00:37:00] Prefill batch. #new-seq: 1, #new-token: 18, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 00:37:02] Decode batch. #running-req: 1, #token: 52, token usage: 0.00, gen throughput (token/s): 4.69, #queue-req: 0


[2025-05-08 00:37:02] Decode batch. #running-req: 1, #token: 92, token usage: 0.00, gen throughput (token/s): 97.35, #queue-req: 0


[2025-05-08 00:37:03] Decode batch. #running-req: 1, #token: 132, token usage: 0.01, gen throughput (token/s): 98.94, #queue-req: 0


[2025-05-08 00:37:03] Decode batch. #running-req: 1, #token: 172, token usage: 0.01, gen throughput (token/s): 100.66, #queue-req: 0


...

[2025-05-08 00:37:21] Decode batch. #running-req: 1, #token: 1932, token usage: 0.09, gen throughput (token/s): 77.64, #queue-req: 0


[2025-05-08 00:37:22] Decode batch. #running-req: 1, #token: 1972, token usage: 0.10, gen throughput (token/s): 65.44, #queue-req: 0


[2025-05-08 00:37:23] Decode batch. #running-req: 1, #token: 2012, token usage: 0.10, gen throughput (token/s): 64.31, #queue-req: 0


[2025-05-08 00:37:23] Decode batch. #running-req: 1, #token: 2052, token usage: 0.10, gen throughput (token/s): 64.50, #queue-req: 0


[2025-05-08 00:37:23] INFO:     127.0.0.1:38730 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "user",
            "content": "Give me the information of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-08 00:37:24] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 00:37:24] Decode batch. #running-req: 1, #token: 44, token usage: 0.00, gen throughput (token/s): 52.55, #queue-req: 0


[2025-05-08 00:37:25] Decode batch. #running-req: 1, #token: 84, token usage: 0.00, gen throughput (token/s): 66.39, #queue-req: 0


[2025-05-08 00:37:25] Decode batch. #running-req: 1, #token: 124, token usage: 0.01, gen throughput (token/s): 64.30, #queue-req: 0


[2025-05-08 00:37:26] Decode batch. #running-req: 1, #token: 164, token usage: 0.01, gen throughput (token/s): 75.87, #queue-req: 0


...

[2025-05-08 00:37:44] Decode batch. #running-req: 1, #token: 1924, token usage: 0.09, gen throughput (token/s): 104.36, #queue-req: 0


[2025-05-08 00:37:44] Decode batch. #running-req: 1, #token: 1964, token usage: 0.10, gen throughput (token/s): 104.33, #queue-req: 0


[2025-05-08 00:37:44] Decode batch. #running-req: 1, #token: 2004, token usage: 0.10, gen throughput (token/s): 104.69, #queue-req: 0


[2025-05-08 00:37:45] Decode batch. #running-req: 1, #token: 2044, token usage: 0.10, gen throughput (token/s): 103.02, #queue-req: 0


[2025-05-08 00:37:45] INFO:     127.0.0.1:38730 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "user",
            "content": "Give me the information of the capital of France.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-08 00:37:45] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 00:37:45] Decode batch. #running-req: 1, #token: 39, token usage: 0.00, gen throughput (token/s): 89.27, #queue-req: 0


[2025-05-08 00:37:46] Decode batch. #running-req: 1, #token: 79, token usage: 0.00, gen throughput (token/s): 107.36, #queue-req: 0


[2025-05-08 00:37:46] Decode batch. #running-req: 1, #token: 119, token usage: 0.01, gen throughput (token/s): 106.49, #queue-req: 0
[2025-05-08 00:37:46] INFO:     127.0.0.1:38730 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-08 00:37:46] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 00:37:46] Decode batch. #running-req: 1, #token: 33, token usage: 0.00, gen throughput (token/s): 97.46, #queue-req: 0


[2025-05-08 00:37:47] Decode batch. #running-req: 1, #token: 73, token usage: 0.00, gen throughput (token/s): 104.83, #queue-req: 0


[2025-05-08 00:37:47] Decode batch. #running-req: 1, #token: 113, token usage: 0.01, gen throughput (token/s): 104.72, #queue-req: 0


[2025-05-08 00:37:48] Decode batch. #running-req: 1, #token: 153, token usage: 0.01, gen throughput (token/s): 104.11, #queue-req: 0


[2025-05-08 00:37:48] Decode batch. #running-req: 1, #token: 193, token usage: 0.01, gen throughput (token/s): 105.59, #queue-req: 0


[2025-05-08 00:37:48] Decode batch. #running-req: 1, #token: 233, token usage: 0.01, gen throughput (token/s): 106.48, #queue-req: 0


[2025-05-08 00:37:49] Decode batch. #running-req: 1, #token: 273, token usage: 0.01, gen throughput (token/s): 101.58, #queue-req: 0


[2025-05-08 00:37:49] Decode batch. #running-req: 1, #token: 313, token usage: 0.02, gen throughput (token/s): 106.04, #queue-req: 0


[2025-05-08 00:37:50] Decode batch. #running-req: 1, #token: 353, token usage: 0.02, gen throughput (token/s): 106.12, #queue-req: 0
[2025-05-08 00:37:50] INFO:     127.0.0.1:38730 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "user",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-05-08 00:37:50] Prefill batch. #new-seq: 1, #new-token: 471, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 00:37:50] Decode batch. #running-req: 1, #token: 495, token usage: 0.02, gen throughput (token/s): 53.85, #queue-req: 0


[2025-05-08 00:37:51] Decode batch. #running-req: 1, #token: 535, token usage: 0.03, gen throughput (token/s): 104.76, #queue-req: 0


[2025-05-08 00:37:51] Decode batch. #running-req: 1, #token: 575, token usage: 0.03, gen throughput (token/s): 104.03, #queue-req: 0


[2025-05-08 00:37:51] Decode batch. #running-req: 1, #token: 615, token usage: 0.03, gen throughput (token/s): 102.46, #queue-req: 0


[2025-05-08 00:37:52] Decode batch. #running-req: 1, #token: 655, token usage: 0.03, gen throughput (token/s): 104.06, #queue-req: 0


[2025-05-08 00:37:52] Decode batch. #running-req: 1, #token: 695, token usage: 0.03, gen throughput (token/s): 103.67, #queue-req: 0


[2025-05-08 00:37:53] Decode batch. #running-req: 1, #token: 735, token usage: 0.04, gen throughput (token/s): 103.40, #queue-req: 0


[2025-05-08 00:37:53] Decode batch. #running-req: 1, #token: 775, token usage: 0.04, gen throughput (token/s): 103.06, #queue-req: 0


[2025-05-08 00:37:53] INFO:     127.0.0.1:38730 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "user",
        "content": "Here is the information of the capital of France in the JSON format.\n",
    }
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-05-08 00:37:54] Prefill batch. #new-seq: 1, #new-token: 19, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-08 00:37:54] Decode batch. #running-req: 1, #token: 29, token usage: 0.00, gen throughput (token/s): 46.12, #queue-req: 0


[2025-05-08 00:37:54] Decode batch. #running-req: 1, #token: 69, token usage: 0.00, gen throughput (token/s): 106.49, #queue-req: 0


[2025-05-08 00:37:55] Decode batch. #running-req: 1, #token: 109, token usage: 0.01, gen throughput (token/s): 106.16, #queue-req: 0


[2025-05-08 00:37:55] Decode batch. #running-req: 1, #token: 149, token usage: 0.01, gen throughput (token/s): 106.56, #queue-req: 0


[2025-05-08 00:37:55] Decode batch. #running-req: 1, #token: 189, token usage: 0.01, gen throughput (token/s): 105.90, #queue-req: 0


...

[2025-05-08 00:38:12] Decode batch. #running-req: 1, #token: 1909, token usage: 0.09, gen throughput (token/s): 103.22, #queue-req: 0


[2025-05-08 00:38:13] Decode batch. #running-req: 1, #token: 1949, token usage: 0.10, gen throughput (token/s): 102.07, #queue-req: 0


[2025-05-08 00:38:13] Decode batch. #running-req: 1, #token: 1989, token usage: 0.10, gen throughput (token/s): 101.98, #queue-req: 0


[2025-05-08 00:38:14] Decode batch. #running-req: 1, #token: 2029, token usage: 0.10, gen throughput (token/s): 103.49, #queue-req: 0


[2025-05-08 00:38:14] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, gen throughput (token/s): 106.48, #queue-req: 0
[2025-05-08 00:38:14] INFO:     127.0.0.1:43832 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so I need to provide the information about the capital of France in JSON format. Hmm, I\'m not entirely sure about all the details, but I\'ll try to think it through.\n\nFirst, I know that the capital of France is Paris. That\'s pretty much a given, right? But I should double-check that. Maybe I can recall any other capitals I know. London is the capital of the UK, Rome is Italy, and maybe Tokyo is Japan\'s. Yeah, Paris seems correct for France.\n\nNow, moving on to the population. I think Paris is a very large city, but I\'m not sure of the exact number. I remember it\'s over 3 million, but I\'m not certain. Maybe around 3.5 million? I should probably look that up, but since I can\'t right now, I\'ll go with 3,500,000 as an estimate.\n\nNext, the area. Paris 

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(text, tokenize=False, add_generation_prompt=True)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-05-08 00:38:14] Prefill batch. #new-seq: 1, #new-token: 3, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 00:38:14] Decode batch. #running-req: 1, #token: 46, token usage: 0.00, gen throughput (token/s): 98.67, #queue-req: 0


[2025-05-08 00:38:15] Decode batch. #running-req: 1, #token: 86, token usage: 0.00, gen throughput (token/s): 106.57, #queue-req: 0


[2025-05-08 00:38:15] Decode batch. #running-req: 1, #token: 126, token usage: 0.01, gen throughput (token/s): 106.94, #queue-req: 0


[2025-05-08 00:38:15] Decode batch. #running-req: 1, #token: 166, token usage: 0.01, gen throughput (token/s): 106.57, #queue-req: 0


...

[2025-05-08 00:38:20] Decode batch. #running-req: 1, #token: 646, token usage: 0.03, gen throughput (token/s): 105.22, #queue-req: 0


[2025-05-08 00:38:20] Decode batch. #running-req: 1, #token: 686, token usage: 0.03, gen throughput (token/s): 105.02, #queue-req: 0


[2025-05-08 00:38:21] Decode batch. #running-req: 1, #token: 726, token usage: 0.04, gen throughput (token/s): 104.92, #queue-req: 0


[2025-05-08 00:38:21] Decode batch. #running-req: 1, #token: 766, token usage: 0.04, gen throughput (token/s): 102.86, #queue-req: 0


[2025-05-08 00:38:21] INFO:     127.0.0.1:45260 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-05-08 00:38:21] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-08 00:38:22] Prefill batch. #new-seq: 3, #new-token: 3, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 00:38:22] Decode batch. #running-req: 3, #token: 41, token usage: 0.00, gen throughput (token/s): 89.04, #queue-req: 0


[2025-05-08 00:38:22] Decode batch. #running-req: 3, #token: 161, token usage: 0.01, gen throughput (token/s): 300.55, #queue-req: 0


[2025-05-08 00:38:23] Decode batch. #running-req: 3, #token: 281, token usage: 0.01, gen throughput (token/s): 303.38, #queue-req: 0


[2025-05-08 00:38:23] Decode batch. #running-req: 3, #token: 401, token usage: 0.02, gen throughput (token/s): 303.31, #queue-req: 0


[2025-05-08 00:38:23] Decode batch. #running-req: 3, #token: 521, token usage: 0.03, gen throughput (token/s): 303.87, #queue-req: 0


[2025-05-08 00:38:24] INFO:     127.0.0.1:47812 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris.\n\nThat's all the information I have.\n\nOkay, so I need to figure out the capital of France. I know that Paris is the capital, but I'm not entirely sure. Let me think about why I think that. I've heard it mentioned a lot, especially in movies and TV shows. People often go there for business or tourism. Also, I remember learning in school that Paris is a major city in France, known for landmarks like the Eiffel Tower and the Louvre Museum. Those places are famous worldwide, which makes me think that Paris is indeed the capital. Maybe I can cross-check this with some other sources or my notes. Wait, I don't have any other information right now, but based on what I know, Paris is the capital of France. I don't recall any other major city in France being referred to as the capital. So, I'm pretty confident that Paris is correct.\n</think>Paris is the capital of F

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-05-08 00:38:24] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-05-08 00:38:24] Decode batch. #running-req: 1, #token: 15, token usage: 0.00, gen throughput (token/s): 240.57, #queue-req: 0


[2025-05-08 00:38:24] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, gen throughput (token/s): 103.61, #queue-req: 0


[2025-05-08 00:38:25] Decode batch. #running-req: 1, #token: 95, token usage: 0.00, gen throughput (token/s): 106.78, #queue-req: 0


[2025-05-08 00:38:25] Decode batch. #running-req: 1, #token: 135, token usage: 0.01, gen throughput (token/s): 103.05, #queue-req: 0


[2025-05-08 00:38:25] Decode batch. #running-req: 1, #token: 175, token usage: 0.01, gen throughput (token/s): 106.62, #queue-req: 0


...

[2025-05-08 00:38:42] Decode batch. #running-req: 1, #token: 1895, token usage: 0.09, gen throughput (token/s): 104.25, #queue-req: 0


[2025-05-08 00:38:42] Decode batch. #running-req: 1, #token: 1935, token usage: 0.09, gen throughput (token/s): 104.53, #queue-req: 0


[2025-05-08 00:38:43] Decode batch. #running-req: 1, #token: 1975, token usage: 0.10, gen throughput (token/s): 104.75, #queue-req: 0


[2025-05-08 00:38:43] Decode batch. #running-req: 1, #token: 2015, token usage: 0.10, gen throughput (token/s): 104.83, #queue-req: 0


[2025-05-08 00:38:43] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, gen throughput (token/s): 107.24, #queue-req: 0
[2025-05-08 00:38:43] INFO:     127.0.0.1:47818 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l 

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-05-08 00:38:43] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 19, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-08 00:38:44] Decode batch. #running-req: 1, #token: 61, token usage: 0.00, gen throughput (token/s): 98.40, #queue-req: 0


[2025-05-08 00:38:44] Decode batch. #running-req: 1, #token: 101, token usage: 0.00, gen throughput (token/s): 106.40, #queue-req: 0


[2025-05-08 00:38:44] Decode batch. #running-req: 1, #token: 141, token usage: 0.01, gen throughput (token/s): 102.05, #queue-req: 0


[2025-05-08 00:38:45] Decode batch. #running-req: 1, #token: 181, token usage: 0.01, gen throughput (token/s): 104.95, #queue-req: 0


...

[2025-05-08 00:38:49] Decode batch. #running-req: 1, #token: 621, token usage: 0.03, gen throughput (token/s): 101.85, #queue-req: 0


[2025-05-08 00:38:50] Decode batch. #running-req: 1, #token: 661, token usage: 0.03, gen throughput (token/s): 101.47, #queue-req: 0


[2025-05-08 00:38:50] Decode batch. #running-req: 1, #token: 701, token usage: 0.03, gen throughput (token/s): 100.73, #queue-req: 0


[2025-05-08 00:38:50] Decode batch. #running-req: 1, #token: 741, token usage: 0.04, gen throughput (token/s): 101.27, #queue-req: 0


[2025-05-08 00:38:51] Decode batch. #running-req: 1, #token: 781, token usage: 0.04, gen throughput (token/s): 100.26, #queue-req: 0
[2025-05-08 00:38:51] INFO:     127.0.0.1:44222 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

[2025-05-08 00:38:51] Child process unexpectedly failed with an exit code 9. pid=3577816


[2025-05-08 00:38:51] Child process unexpectedly failed with an exit code 9. pid=3577742


## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.41s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.34s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.35s/it]



### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure, here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Tiananmen Square",
    "Forbidden City",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text:  the user provided some facts but needs a more detailed response. Let's see:

The capital of France is Paris. Paris is the capital city of France, located in the northern part of the country. It is a metropolis that has been its official capital since 1998. Paris has been the capital of France since 18th century. The current mayor of Paris is Invoice Z. Leblond. Paris is a city of culture, art, and history, with a rich history dating back to ancient times. It is a major political, economic, and cultural center of Western Europe and, by extension, the world
Prompt: Give me the information of the capital of Germany.
Generated text: 
The capital of Germany is Berlin.

The capital of Germany is Berlin.

The capital of Germany is Berlin.

The capital of Germany is Berlin.

The capital of Berlin is Germany.

The capital of Germany is Berlin.

The capital of Berlin is Germany.

The capital of Berlin is Germany.

The cap

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  1. the population, 2. the major industries, 3. the cultural significance, and 4. the main transportation systems.

Please put it in an organized format with clear headings.
Certainly! Here's an organized overview of London as a major global city, covering population, major industries, cultural significance, and main transportation systems:

---

### 1. Population
- **Metropolitan Area**: Approximately 9 million people.
- **Greater伦敦**: Around 34 million people.

### 2. Major Industries
- **Finance**: London is a global hub for finance, banking, and investment.
- **Technology**:
Prompt: Please provide information about Paris as a major global city:
Generated text:  its location, economic status, cultural significance, and the infrastructure and transportation system.

 Paris is one of the most well-known cities in the world, with a rich history and a vibrant cultural scene. It is located in western 

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜User｜>Here is the information of the capital of France in the JSON format.
<｜Assistant｜><think>

Generated text: Okay, I need to provide a JSON object containing information about the capital of France, which is Paris. Let me start by recalling what I know about Paris.

First, Paris is the administrative capital, but I think it's also the political and cultural center. I'm pretty sure its population is around 2 million. The area is about 105 square kilometers. It's located in the northern part of the country, specifically in the Île-de-France region.

Paris is famous for landmarks like the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. It's also a major transportation hub with the Paris地铁 and航空 airport. The city has a rich history, with the Treaty of Westphalia making it the capital after the French Revolution. There's also the iconic Le Marais district and the Île Saint-Louis as landmarks.

Putting this all together, I'll structure the JSON w

In [19]:
llm.shutdown()